# S2S Forecast Engine Verification

This notebook verifies the implementation of the Damped Persistence forecast engine using real ERA5 data downloaded via CDSAPI.

In [ ]:
import os
import sys
import pandas as pd
import xarray as xr
import matplotlib.pyplot as plt
import numpy as np

# Add project root to path to import lib
sys.path.append(os.path.abspath(".."))

from lib.forecast.engine import forecast_damped_persistence
from lib.indices.construct import OUT_ALL

print(f"Data Path: {OUT_ALL}")

: 

## 1. Load Data
Load the locally downloaded ERA5 NetCDF file.

In [ ]:
if not os.path.exists(OUT_ALL):
    print("Data file not found! Please run the download script first.")
else:
    ds = xr.open_dataset(OUT_ALL)
    print(ds)

## 2. Select a Location
Choose a coordinate (e.g., Santiago) to visualize and forecast.

In [ ]:
lat = -33.4
lon = -70.6

# Select nearest point
ds_point = ds.sel(latitude=lat, longitude=lon, method="nearest")

# Extract time series
t2m = ds_point["t2m"]
time = ds_point["time"]

plt.figure(figsize=(12, 6))
plt.plot(time, t2m, label="ERA5 T2M")
plt.title(f"Temperature at {lat}, {lon}")
plt.legend()
plt.grid(True)
plt.show()

## 3. Run Forecast
Use the `forecast_damped_persistence` engine to predict the next 24 months.

In [ ]:
# Prepare inputs
current_val = float(t2m.isel(time=-1).values)
current_date = pd.Timestamp(time.isel(time=-1).values)

# Mock Climatology (since we don't have the full 30-year clim file yet)
# Ideally, we would load this from the climatology file generated by construct.py
clim_means = [288.0] * 12 # Placeholder
clim_stds = [2.0] * 12    # Placeholder

print(f"Current Date: {current_date}")
print(f"Current Value: {current_val}")

# Run Engine
forecast = forecast_damped_persistence(
    current_value=current_val,
    current_date=current_date,
    climatology_means=clim_means,
    climatology_stds=clim_stds,
    horizon_months=24
)

# Convert to DataFrame for plotting
df_fcst = pd.DataFrame(forecast)
df_fcst["date"] = pd.to_datetime(df_fcst["date"])
df_fcst.head()

## 4. Visualize Forecast
Plot the historical data and the forecast with probability bands.

In [ ]:
plt.figure(figsize=(12, 6))

# Plot History (Last 2 years)
history_window = t2m.sel(time=slice(current_date - pd.DateOffset(months=24), current_date))
plt.plot(history_window.time, history_window.values, label="History", color="black")

# Plot Forecast
plt.plot(df_fcst["date"], df_fcst["mean"], label="Forecast Mean", color="blue", linestyle="--")
plt.fill_between(df_fcst["date"], df_fcst["p05"], df_fcst["p95"], color="blue", alpha=0.1, label="90% Confidence")
plt.fill_between(df_fcst["date"], df_fcst["p50"] - 1, df_fcst["p50"] + 1, color="blue", alpha=0.2, label="Median Zone")

plt.title("S2S Forecast: Damped Persistence")
plt.legend()
plt.grid(True)
plt.show()